# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [ ]:
data.rename(columns={'Description':'ProductName'}, inplace=True)

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [ ]:
grouped = data.groupby(['CustomerID', 'ProductName']).sum()
grouped

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [ ]:
matrix = grouped.pivot_table('Quantity', 'ProductName', 'CustomerID', aggfunc='sum', fill_value = 0)
matrix

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [ ]:
# I need to transpose the matrix, otherwise I get the distance for products, no customers.
# First applying pdist, gives an 1D array.
# Then applying squareform to turn it into a squareform
# Finally convert it into a DataFrame

dist_matrix = pd.DataFrame(squareform(pdist(matrix.T)), index=matrix.columns, columns=matrix.columns)
dist_matrix

In [ ]:
# The distances I have doesn't tell me much. I will normalize to a value between 0 and 1,
# and inverse them: The closer to 1, the more similar they are

dist_norm = pd.DataFrame(1/(1 + dist_matrix))
dist_norm

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [ ]:
Top5_cust200 = dist_norm[17976].sort_values(ascending = False).head(6)
Top5_cust200

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [ ]:
# I select index from 1 because I don't want to get the first input, 
# as it is the customer itself

similar = grouped.loc[(Top5_cust200.index[1:],)]
similar

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [ ]:
agg_similar = similar.groupby('ProductName')[['Quantity']].sum()\
                .sort_values(by = 'Quantity', ascending = False)
agg_similar

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [ ]:
products = pd.concat([agg_similar, matrix[17976]], axis=1, sort=False)
products.rename(columns = {17976:'Cust_200'}, inplace = True)
products

In [ ]:
Top5rec = products.query('Quantity > 0 and Cust_200 == 0').head(5)
Top5rec

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [ ]:
recommendations = {}
unique_ID = dist_norm.columns.unique()

In [ ]:
for customer in unique_ID:
    head = dist_norm[customer].sort_values(ascending = False).head(6)
    similar = grouped.loc[(head.index[1:],)]
    agg_similar = similar.groupby('ProductName')[['Quantity']].sum()\
                .sort_values(by = 'Quantity', ascending = False)
    products = pd.concat([agg_similar, matrix[customer]], axis=1, sort=False)
    products.rename(columns = {customer:'customer'}, inplace = True)
    recommendations[customer] = list(products.query('Quantity > 0 and customer == 0').head(5).index)
    

In [ ]:
recommendations

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [ ]:
recommendations_df = pd.DataFrame.from_dict(recommendations, orient='index', 
                                columns=['rec1', 'rec2', 'rec3', 'rec4', 'rec5'])
recommendations_df

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [ ]:
data.Quantity.unique()